Sampling from GPs
=================

In [1]:
import numpy as np, scipy
%matplotlib
from matplotlib import pyplot as plt
from matplotlib import animation
import itertools
import GPy

from matplotlib import rc
from matplotlib import cm
rc('text', usetex=True)

Using matplotlib backend: Qt4Agg


# Choose here whether to sample from a fitted GP or zero mean samples:

In [2]:
# Switch for wether to use the data from toy_rbf to plot. 
# if true, we wont use the data, if false we will
zero_mean = False

# switch for 1d or 2d
one_d = True

In [3]:
# First set up the figure, the axis, and the plot element we want to animate
plt.close('all')
if one_d:
    fig = plt.figure(figsize=(8,4))
    vl_list = [(1,10), (1,1), (1,.2)]
    colors = ["#BBCDD9","#2A3F54","#5A9ED2","#687C8E"] # tahnks to http://tools.medialab.sciences-po.fr/iwanthue/index.php

    m = GPy.examples.regression.toy_rbf_1d_50(plot=False)
    #m.plot(ax=ax)
else:
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D
    vl_list = [(1,1)]
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

In [4]:
if one_d:
    ks = [GPy.kern.RBF(1, variance=v, lengthscale=l) for v,l in vl_list]
    N = 100
    mi, ma = m.X.min(), m.X.max()
    r = m.X.max() - m.X.min()
    x = np.linspace(mi-r, ma+r, N)[:,None]

    ax = plt.axes(xlim=(mi-r, ma+r), ylim=(-2, 2))
    ax.set_frame_on(False)
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)

    if zero_mean:
        # Plot just samples:
        l_nothing = GPy.plotting.matplot_dep.base_plots.gpplot(x, np.zeros_like(x), np.zeros_like(x)+2, np.zeros_like(x)-2, ax=ax)[0][0]
        ax.set_ylim(-3,3)
    else:
        # Plot with mean:
        l_nothing = m.plot(ax=ax, plot_limits=(mi-r,ma+r))['gpplot'][0][0]

    lines = [ax.plot(np.linspace(-1,1,100), 
                     np.sin(np.linspace(-1,1,100)-np.random.randn()), lw=.8, 
                     color=c, 
                     label=r'$\ell={:.2G}$'.format(l))[0] for (v,l),c in itertools.izip(vl_list, colors)]

    #legend = ax.legend(
    if zero_mean:
        legend = ax.legend(bbox_to_anchor=(0., 1.0, 1., .1), loc=3,
               ncol=4, mode="expand", borderaxespad=0.)
else:
    ks = [GPy.kern.Matern32(2, lengthscale=2)]
    X, Y = np.mgrid[-5:5:40j, -5:5:40j]
    N = X.size
    x = np.c_[X.flat, Y.flat]
    Ks = [ks[0].K(x)]
    Z = np.random.multivariate_normal(np.zeros(x.shape[0]), Ks[0])
    ax.plot_surface(X, Y, Z.reshape(X.shape[0], Y.shape[1]), cmap=cm.jet, rstride=1, cstride=1)
    
fig.tight_layout(rect=(0, 0, 1, .91))


In [5]:
if one_d:
    if zero_mean:
        mus = [np.zeros_like(x) for _ in vl_list]
        Ks = [k.K(x) for k in ks]
    else:
        mu, K = m._raw_predict(x, full_cov=True)
        mus = [mu for _ in vl_list]
        Ks = [K for _ in vl_list]

In [6]:
#plt.matshow(Ks[0])

#ax.plot(x,np.zeros(N),color='k',lw=2,ls='--')
#ax.fill_between(x[:,0],np.ones(N)*2,-np.ones(N)*2,color='k',alpha=.2,linestyle='--')

Rs = [GPy.util.linalg.pdinv(K+np.eye(N)*1e-8)[1] for K in Ks]

In [7]:
def exp_map_sphere(mu, E):
    theta = np.sqrt((E**2).sum(0))[None]
    M = mu * np.sin(theta)
    M = M + (E * (np.cos(theta))/theta)
    M[:, np.abs(theta[:,0])<=1e-7] = mu
    return M

def exp_map(mu, E):
    theta = np.sqrt((E**2).sum(0))[None]
    M = mu * np.sin(theta)
    M = M + (E * (np.cos(theta))/theta)
    M[:, np.abs(theta[:,0])<=1e-7] = mu
    return M

def gp_animate(d,n):
    u = np.random.normal(0,1,size=(N,1))
    r = np.sqrt((u**2).sum())
    u /= r
    t = np.random.normal(0,1,size=(N,1))
    t = t - (t.T.dot(u)).dot(u.T).T
    t /= np.sqrt((t**2).sum())
    start = np.random.uniform(0,2*np.pi)
    T = np.linspace(start, start+2*np.pi, n)[None] * t
    return r*exp_map(u, np.linspace(0.001, 2*np.pi, n)[None] * t)

frames = 200
draws = [gp_animate(N,frames) for _ in vl_list]

In [8]:
# initialization function: plot the background of each frame
if one_d:
    def init():
        for line in lines:
            line.set_data([], [])
        return lines
else:
    def init():
        del ax.collections[:]
        return [ax.plot_surface(X, Y, Z.reshape(X.shape[0], Y.shape[1]), cmap=cm.jet, rstride=1, cstride=1)]

In [9]:
# animation function.  This is called sequentially
s = [R.dot(draw) for R,draw in zip(Rs,draws)]
if one_d:
    def animate(i):
        for sprime, mu, line in zip(s, mus, lines):
            #print y[:,i].shape, x.shape
            line.set_data(x[:,0], mu[:,0]+sprime[:,i])
        return lines
else:
    def animate(i):
        del ax.collections[:]
        ax.set_zlim(-5,5)
        return [ax.plot_surface(X, Y, s[0][:,i].reshape(X.shape[0], Y.shape[1]), cmap=cm.jet, rstride=1, cstride=1)]
animate(100)
plt.draw()

In [12]:
# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=frames, interval=20, blit=True)
#plt.show()

In [27]:
# save the animation as an mp4.  This requires ffmpeg or mencoder to be
# installed.  The extra_args ensure that the x264 codec is used, so that
# the video can be embedded in html5.  You may need to adjust this for
# your system: for more information, see
# http://matplotlib.sourceforge.net/api/animation_api.html
for i in xrange(frames):
    if not one_d:
        animate(i)
        fig.canvas.draw()
        fig.savefig('../images/gp_animation/2D/matern_anim{:0>3}.png'.format(i))        
    elif zero_mean:
        animate(i)
        fig.canvas.draw()
        fig.savefig('../images/gp_animation/zero_mean/anim{:0>3}.pdf'.format(i))
        #anim.save('gp_animation_zero_mean.mp4', fps=30, extra_args=['-vcodec', 'libx264'])
    else:
        animate(i)
        fig.canvas.draw()
        fig.savefig('../images/gp_animation/mean/anim{:0>3}.pdf'.format(i))
        #anim.save('gp_animation_mean.mp4', fps=30, extra_args=['-vcodec', 'libx264'])

In [160]:
if zero_mean:
    fig.savefig('gp_animation_zero_mean_poster.pdf')
else:
    fig.savefig('gp_animation_mean_poster.pdf')